In [60]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import mysql.connector
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from user_agent import generate_user_agent, generate_navigator
from selenium.common.exceptions import UnexpectedAlertPresentException
import requests
import datetime

In [65]:
def movie_crawling(webdriver_path, service, driver, id):
    webdriver_path = webdriver_path
    service = service
    driver = driver
    try:
        url = f'http://cine21.com/movie/info/?movie_id={id}'
        driver.get(url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        sub_info = soup.find_all('p', class_='sub_info')
        sub_info_list = [i.text.strip().replace('\n', ' ').replace('\t', ' ') for i in sub_info]
        ko_name = soup.find('p', class_='tit').text
        en_name = soup.find('p', class_='tit_eng').text.strip().split('\n')[-1]
        story = soup.find('div', class_='story').text.strip().replace('\t', '').replace('\n', ' ').replace('  ', ' ')
        if story == '':
            story = None
        year = sub_info_list[0].split()[0]
        country = sub_info_list[0].split()[1]
        poster_link = soup.find('div', class_='mov_poster').find('img')['src']
        if poster_link == 'https://image.cine21.com/resize//noimg[X230,330].jpg':
            poster_link = None

        try:
            if sub_info_list[2][:3] == '개봉일' and len(sub_info_list[2][6:16]) == 10:
                pd.to_datetime(sub_info_list[2][6:16])
                release_date = sub_info_list[2][6:16]
            else:
                release_date = None
        except:
            release_date = None
    
        if sub_info_list[2][17:21] == '누적관객':
            audience = int(sub_info_list[2][24:].replace(',', '').replace('명', '').replace(',', ''))
        else:
            audience = None
        
        if sub_info_list[1].split('상영시간 : ')[0] != '':
            genre = sub_info_list[1].split('상영시간 : ')[0][:-1]
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])
        else:
            genre = None
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])

        if sub_info_list[3][0:2] == '감독':
            director = sub_info_list[3][18:]
        else:
            director = None

        for i in range(1, 6):
            if soup.find('span', class_=f'grade00{i}') != None:
                grade = soup.find('span', class_=f'grade00{i}').text
                break
            elif i == 5:
                grade = None

        return [id, ko_name, en_name, year, country, grade, genre, running_time, release_date, audience, director, story, poster_link]
    
    except UnexpectedAlertPresentException as e1:
        return e1.alert_text
    
    except IndexError as e2:
        return '영화 정보 불충분'
    
    except AttributeError as e3:
        return '일시적으로 서비스 장애가 발생하였습니다.'

In [66]:
create_table_query = """
CREATE TABLE IF NOT EXISTS movies (
    id INT PRIMARY KEY,
    ko_name VARCHAR(255),
    en_name VARCHAR(255),
    country VARCHAR(255),
    grade VARCHAR(255),
    genre VARCHAR(255),
    runtime INT,
    release_date DATE,
    audience INT,
    director VARCHAR(255),
    story VARCHAR(255),
    poster_link VARCHAR(255))"""

In [63]:
# # My SQL 데이터베이스 연결
# conn = mysql.connector.connect(
#     host = 'localhost',
#     user = 'root',
#     password = 'signcity114',
#     database = 'MyDB'
# )

# cursor = conn.cursor()

# cursor.execute(create_table_query)
# conn.commit()
# print("테이블이 성공적으로 생성되었습니다")

# conn.close()

In [76]:
# My SQL 데이터베이스 연결
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'signcity114',
    database = 'MyDB'
)

# 연결 확인
if conn.is_connected():
    print("MySQL에 성공적으로 연결되었습니다")
else:
    print("MySQL 연결 실패")

cursor = conn.cursor()

webdriver_path = '/Users/anhyojun/WebDriver/chromedriver'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

insert_query = """
INSERT INTO movie
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for i in range(59234, 60000):
    data = movie_crawling(webdriver_path, service, driver, i)
    if data == '존재하지 않는 영화입니다.':
        print(data)
        continue
    elif data == '성인용 컨텐츠입니다. 로그인 해 주세요.':
        print(data)
        continue
    elif data == '영화 정보 불충분':
        print(data)
        continue
    elif data == '일시적으로 서비스 장애가 발생하였습니다.':
        print(data)
        continue
    cursor.execute(insert_query, data)
    conn.commit()
    print(data)

MySQL에 성공적으로 연결되었습니다
[59234, '이터널 로드', 'The Eternal Road', '2017', '핀란드,에스토니아,스웨덴', '15세이상관람가', '드라마', 99, '2022-05-27', None, '안티 주시 아닐라', '만명이 넘는 미국 이주노동자들 대학살 사건! 1930년대, 냉전시대의 이념 갈등이라는 혼란에 휘말린 ‘주시 케톨라’ 소위 ‘영원한 길’이라 불리는 소비에트 국경 지역에 강제로 버려진다. 소비에트에 피랍된 ‘주시 케톨라’는 자신의 가족들에게 돌아가기 위해 그의 목숨을 건 탈출을 시도하는데…', 'https://image.cine21.com/resize/cine21/poster/2022/0519/59234_6285b37207151[X230,330].jpg']
[59235, '쥬라기 리턴즈', 'Hatched', '2021', '영국', '12세이상관람가', '액션, SF', 84, '2022-05-24', None, '스콧 제프리 레베카 매슈스', '금기의 실험, 그 대가는 죽음이다 깨워선 안 될 최강 살인 공룡들이 눈 뜬다! ‘사이먼’은 유전자 조작으로 랩터와 티라노들을 부활시키지만, 하루가 다르게 거대해지는 진화 속도를 감당하지 못하고 무참히 잡아먹힌다. 한편 연락이 끊긴 오빠 ‘사이먼’의 집에 찾아온  ‘조슬린’과 가족들은 눈앞에서 살인 공룡들을 마주하게 된다. 이 사실을 알게 된 최정예 특수부대는 세상을 혼란에 빠트릴 공룡들을 막기 위해 마지막 전투를 준비하고,  이 상황 속에서 ‘조슬린’과 가족들은 목숨을 건 탈출을 하려 하는데… 인간과 공룡의 마지막 승부가 시작된다!', 'https://image.cine21.com/resize/cine21/poster/2022/0511/59235_627b64ae4aae0[X230,330].jpg']
[59236, '옴진리교: 지하철 사린 사건과 나', 'ME AND THE CULT LEADER', '2020', '일본', None, '다큐멘터리', 114, None, N

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [54]:
webdriver_path = '/Users/anhyojun/WebDriver/chromedriver'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)
movie_crawling(webdriver_path, service, driver, 415)

[415,
 '언지프',
 'Unzipped',
 '미국',
 '15세이상관람가',
 '다큐멘터리',
 73,
 None,
 7903,
 '더글러스 키브',
 '흥미진진하고 즐거움을 주는 매우 사적인 패션 디자이너 마즈라히는 Nanlook of the North, Samantha Stevens 등 다양한 문화계의 우상들에게 영감을 얻으면서 좌절과 기쁨과 공포로 가득한 모델들이 걸어다니는 진짜 패션쇼를 만들어내기 위해 분주하다. 그는 중요한 잡지 편집자를 위해 자신의 디자인을 재검토하고 루브르에서 기록으로 남아 있는 패션 콜렉션을 보기 위해 파리로 여행하기도 한다.  쇼가 가까워지자 그는 디자인에 생명을 불어넣고, 케이트 못, 린다 에반젤리스타, 신디 크로포드 같은 세계 톱모델들과 계약을 체결하는 과정을 거치면서 유명인사들과 비평가들로 들어찬 쇼장에서 대담하고 복잡미묘하게 신경을 곤두서게 하는 쇼를 진행한다.',
 'https://image.cine21.com/resize/cine21/poster/2005/0615/M0010197_[X230,330].jpg']